In [ ]:
import torch

data = torch.load("tmp/nurec_sample/checkpoint.ckpt")

In [ ]:
state_dict = dict(data["state_dict"])

for k in state_dict:
    print(k)

"""
model.gaussians_nodes.dynamic_rigids.positions
model.gaussians_nodes.dynamic_rigids.rotations
model.gaussians_nodes.dynamic_rigids.scales
model.gaussians_nodes.dynamic_rigids.densities
model.gaussians_nodes.dynamic_rigids.extra_signal
model.gaussians_nodes.dynamic_rigids.features_albedo
model.gaussians_nodes.dynamic_rigids.features_specular
model.gaussians_nodes.dynamic_rigids.gaussian_cuboid_ids
model.gaussians_nodes.dynamic_rigids.n_active_features
model.gaussians_nodes.dynamic_rigids._extra_state
model.gaussians_nodes.dynamic_rigids.tracks_calib.tracks_delta_q
model.gaussians_nodes.dynamic_rigids.tracks_calib.tracks_delta_t
"""

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

means = state_dict["model.gaussians_nodes.background.positions"].numpy()  # [N, 3]
print(means.shape)
print(np.std(means, axis=0))
plt.scatter(means[:, 0], means[:, 1])

In [ ]:
state_dict["model.gaussians_nodes.dynamic_rigids.gaussian_cuboid_ids"].unique()

In [ ]:
import json
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt

seq_data = json.loads(Path("tmp/nurec_sample/datasource_summary.json").read_text())
tracks_data = next(iter(seq_data["sequence_tracks_dynamic"].values()))["tracks_data"]

centers = []
for track_poses in tracks_data["tracks_poses"]:
    for pose in track_poses:
        centers.append(np.array(pose)[:3])
centers = np.stack(centers)

# pose: <x,y,z, qw, qx, qy, qz>
plt.scatter(centers[:, 0], centers[:, 1])

In [ ]:
# Extract background and road gaussian positions
bg_pos = state_dict["model.gaussians_nodes.background.positions"].numpy()  # [N_bg, 3]
road_pos = state_dict["model.gaussians_nodes.road.positions"].numpy()  # [N_road, 3]

# Extract depth values for coloring (z-coordinate)
bg_depth = bg_pos[:, 2]
road_depth = road_pos[:, 2]

# Calculate display range based on dynamic object trajectory centers
x_min, x_max = centers[:, 0].min(), centers[:, 0].max()
y_min, y_max = centers[:, 1].min(), centers[:, 1].max()

# Add some padding to the display range
padding = 5.0
x_min -= padding
x_max += padding
y_min -= padding
y_max += padding

# Create the plot
fig, ax = plt.subplots(figsize=(12, 10))

# Plot background gaussians with depth coloring
scat_bg = ax.scatter(
    bg_pos[:, 0], bg_pos[:, 1], c=bg_depth, cmap="viridis", s=1, alpha=0.3, label="Background Gaussians"
)

# Plot road gaussians with depth coloring
scat_road = ax.scatter(
    road_pos[:, 0], road_pos[:, 1], c=road_depth, cmap="viridis", s=1, alpha=0.5, label="Road Gaussians"
)

# Plot dynamic object trajectory centers
ax.scatter(
    centers[:, 0],
    centers[:, 1],
    c="red",
    s=20,
    marker="o",
    alpha=0.8,
    label="Dynamic Object Centers",
    edgecolors="black",
    linewidth=0.5,
)

# Set display range to match dynamic object range
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)

# Add colorbar for depth
cbar = plt.colorbar(scat_bg, ax=ax)
cbar.set_label("Depth (z)", rotation=270, labelpad=15)

# Add labels and title
ax.set_xlabel("X (meters)")
ax.set_ylabel("Y (meters)")
ax.set_title("Vertical Projection: Background & Road Gaussians with Dynamic Object Trajectories")
ax.legend(loc="upper right")
ax.grid(True, alpha=0.3)
ax.set_aspect("equal", adjustable="box")

plt.tight_layout()
plt.show()

In [ ]:
# Load camera trajectory data from datasource_summary.json
import json

import numpy as np

# Load datasource_summary.json
with open("tmp/nurec_sample/datasource_summary.json") as f:
    datasource_data = json.load(f)

# Navigate to rig_trajectories
trajectory = datasource_data["rig_trajectories"]["rig_trajectories"][0]

# Extract camera centers from T_rig_world transformation matrices
camera_centers = []
for T in trajectory["T_rig_worlds"]:
    T_mat = np.array(T)
    camera_centers.append(T_mat[:3, 3])  # Extract translation (camera center position)

T_base = np.array(datasource_data["rig_trajectories"]["world_to_nre"]["matrix"])

camera_centers = np.array(camera_centers) @ T_base[:3, :3].T + T_base[:3, 3]

print(f"Loaded {len(camera_centers)} camera trajectory poses from datasource_summary.json")
print(
    f"Camera centers range: X[{camera_centers[:, 0].min():.1f}, {camera_centers[:, 0].max():.1f}], "
    f"Y[{camera_centers[:, 1].min():.1f}, {camera_centers[:, 1].max():.1f}], "
    f"Z[{camera_centers[:, 2].min():.1f}, {camera_centers[:, 2].max():.1f}]"
)

In [ ]:
# Visualize background points with camera trajectory centers
fig, ax = plt.subplots(figsize=(14, 10))

# Plot background gaussians with depth coloring
scat_bg = ax.scatter(bg_pos[:, 0], bg_pos[:, 1], c=bg_depth, cmap="Blues", s=2, alpha=0.4, label="Background Gaussians")

# Plot road gaussians with depth coloring
scat_road = ax.scatter(
    road_pos[:, 0], road_pos[:, 1], c=road_depth, cmap="Greys", s=2, alpha=0.5, label="Road Gaussians"
)

# Plot camera trajectory centers with direction arrows
ax.scatter(
    camera_centers[:, 0],
    camera_centers[:, 1],
    c="red",
    s=50,
    marker="o",
    alpha=0.9,
    label="Camera Centers",
    edgecolors="darkred",
    linewidth=1.5,
    zorder=5,
)

# Add trajectory direction arrows (show every 10th pose to avoid clutter)
arrow_step = 10
for i in range(0, len(camera_centers) - arrow_step, arrow_step):
    start = camera_centers[i]
    end = camera_centers[i + arrow_step]
    direction = end - start
    ax.arrow(
        start[0],
        start[1],
        direction[0],
        direction[1],
        head_width=2,
        head_length=3,
        fc="orange",
        ec="orange",
        alpha=0.6,
        zorder=4,
        linewidth=1.5,
    )

# Mark start and end points
ax.scatter(
    camera_centers[0, 0],
    camera_centers[0, 1],
    c="green",
    s=200,
    marker="^",
    label="Start",
    edgecolors="darkgreen",
    linewidth=2,
    zorder=6,
)
ax.scatter(
    camera_centers[-1, 0],
    camera_centers[-1, 1],
    c="purple",
    s=200,
    marker="s",
    label="End",
    edgecolors="darkmagenta",
    linewidth=2,
    zorder=6,
)

# Set display range based on camera trajectory
x_min, x_max = camera_centers[:, 0].min(), camera_centers[:, 0].max()
y_min, y_max = camera_centers[:, 1].min(), camera_centers[:, 1].max()
padding = max((x_max - x_min), (y_max - y_min)) * 0.1
ax.set_xlim(x_min - padding, x_max + padding)
ax.set_ylim(y_min - padding, y_max + padding)

# Add colorbar for depth
cbar = plt.colorbar(scat_bg, ax=ax)
cbar.set_label("Depth (z)", rotation=270, labelpad=15, fontsize=12)

# Add labels and title
ax.set_xlabel("X (meters)", fontsize=12)
ax.set_ylabel("Y (meters)", fontsize=12)
ax.set_title("Background Gaussians with Camera Trajectory Centers", fontsize=14, fontweight="bold")
ax.legend(loc="best", framealpha=0.9)
ax.grid(True, alpha=0.3, linestyle="--")
ax.set_aspect("equal", adjustable="box")

plt.tight_layout()
plt.show()

# Print statistics
print("\n=== Statistics ===")
print(f"Background gaussians: {len(bg_pos):,}")
print(f"Road gaussians: {len(road_pos):,}")
print(f"Camera trajectory poses: {len(camera_centers)}")
print(f"Trajectory length: {np.sum(np.linalg.norm(camera_centers[1:] - camera_centers[:-1], axis=1)):.1f} meters")